In [16]:
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt
from IPython.display import HTML

# We also set the logging level so that we get some feedback from the API
import logging
logging.basicConfig(level=logging.INFO)

In [17]:
# Quick hack to put us in the icenet-pipeline folder,
# assuming it was created as per 01.cli_demonstration.ipynb
import os
if os.path.exists("6_tensorflow_normal_run.ipynb"):
    os.chdir("../notebook-pipeline")
print("Running in {}".format(os.getcwd()))

%matplotlib inline

Running in /data/hpcdata/users/bryald/git/icenet/notebook-pipeline


### Download required datasets

In [18]:
# Download masks
!icenet_data_masks south

[18-12-23 18:33:41 :INFO    ] - siconca ice_conc_sh_ease2-250_cdr-v2p0_200001021200.nc already exists
[18-12-23 18:33:42 :INFO    ] - Saving ./data/masks/south/masks/active_grid_cell_mask_01.npy
[18-12-23 18:33:42 :INFO    ] - siconca ice_conc_sh_ease2-250_cdr-v2p0_200002021200.nc already exists
[18-12-23 18:33:42 :INFO    ] - Saving ./data/masks/south/masks/active_grid_cell_mask_02.npy
[18-12-23 18:33:42 :INFO    ] - siconca ice_conc_sh_ease2-250_cdr-v2p0_200003021200.nc already exists
[18-12-23 18:33:42 :INFO    ] - Saving ./data/masks/south/masks/active_grid_cell_mask_03.npy
[18-12-23 18:33:42 :INFO    ] - siconca ice_conc_sh_ease2-250_cdr-v2p0_200004021200.nc already exists
[18-12-23 18:33:42 :INFO    ] - Saving ./data/masks/south/masks/active_grid_cell_mask_04.npy
[18-12-23 18:33:42 :INFO    ] - siconca ice_conc_sh_ease2-250_cdr-v2p0_200005021200.nc already exists
[18-12-23 18:33:42 :INFO    ] - Saving ./data/masks/south/masks/active_grid_cell_mask_05.npy
[18-12-23 18:33:42 :INFO 

In [19]:
# Download climate data - ERA5 reanalysis data
!icenet_data_era5 south -d --vars uas,vas,tas,zg --levels ',,,500|250' 2020-1-1 2020-4-30

[18-12-23 18:33:49 :INFO    ] - ERA5 Data Downloading
[18-12-23 18:33:49 :WARNING ] - !!! Deletions of temp files are switched off: be careful with this, you need to manage your files manually
[18-12-23 18:33:49 :INFO    ] - Building request(s), downloading and daily averaging from ERA5 API
[18-12-23 18:33:49 :INFO    ] - Processing single download for uas @ None with 121 dates
[18-12-23 18:33:49 :INFO    ] - Processing single download for vas @ None with 121 dates
[18-12-23 18:33:49 :INFO    ] - Processing single download for tas @ None with 121 dates
[18-12-23 18:33:49 :INFO    ] - Processing single download for zg @ 500 with 121 dates
[18-12-23 18:33:49 :INFO    ] - Processing single download for zg @ 250 with 121 dates
[18-12-23 18:33:49 :INFO    ] - No requested dates remain, likely already present
[18-12-23 18:33:49 :INFO    ] - No requested dates remain, likely already present
[18-12-23 18:33:49 :INFO    ] - No requested dates remain, likely already present
[18-12-23 18:33:49 :I

In [20]:
# Download sea ice concentration (%) from OSI-SAF
!icenet_data_sic south -d 2020-1-1 2020-4-30

[18-12-23 18:33:51 :INFO    ] - OSASIF-SIC Data Downloading
[18-12-23 18:33:51 :INFO    ] - Downloading SIC datafiles to .temp intermediates...
[18-12-23 18:33:52 :INFO    ] - Excluding 121 dates already existing from 121 dates requested.
[18-12-23 18:34:02 :INFO    ] - Existing file needs concatenating: ./data/osisaf/south/siconca/2020.nc -> ./data/osisaf/south/siconca/old.2020.nc
[18-12-23 18:34:04 :INFO    ] - Saving ./data/osisaf/south/siconca/2020.nc
[18-12-23 18:34:04 :INFO    ] - Opening for interpolation: ['./data/osisaf/south/siconca/2020.nc']
[18-12-23 18:34:04 :INFO    ] - Processing 0 missing dates


### Process above data downloads - normalising to use as inputs for UNet

This creates loader.{name}.json.

Also creates train, val, test splits based on CLI arguments within the loader json file.

In [21]:
# Process ERA5
!icenet_process_era5 notebook_tf_data south \
    -ns 2020-1-1 -ne 2020-3-31 -vs 2020-4-3 -ve 2020-4-23 -ts 2020-4-1 -te 2020-4-2 \
    -l 1 --abs uas,vas --anom tas,zg500,zg250

# Process SIC
!icenet_process_sic notebook_tf_data south \
    -ns 2020-1-1 -ne 2020-3-31 -vs 2020-4-1 -ve 2020-4-20 -ts 2020-4-1 -te 2020-4-2 \
    -l 1 --abs siconca

!icenet_process_metadata notebook_tf_data south

2023-12-18 18:34:07.177712: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 18:34:07.177798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 18:34:07.179024: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[18-12-23 18:34:11 :INFO    ] - Got 91 dates for train
[18-12-23 18:34:11 :INFO    ] - Got 21 dates for val
[18-12-23 18:34:11 :INFO    ] - Got 2 dates for test
[18-12-23 18:34:11 :INFO    ] - Creating path: ./processed/notebook_tf_data/era5
[18-12-23 18:34:11 :INFO    ] - Processing 91 dates for train category
[18-12-23 18:34:11 :INFO    ] - Including lag of 1 d

Creates `dataset_config.{name}.json`, and cached tfrecords dataset for training.

If running in config-only model, will create json file, but not cached dataset (i.e., for just prediction)

In [22]:
!icenet_dataset_create -l 1 -fd 7 -ob 2 -w 4 notebook_tf_data south

2023-12-18 18:37:01.860011: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 18:37:01.860087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 18:37:01.861441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[18-12-23 18:37:03 :INFO    ] - Got 0 dates for train
[18-12-23 18:37:03 :INFO    ] - Got 0 dates for val
[18-12-23 18:37:03 :INFO    ] - Got 0 dates for test
[18-12-23 18:37:03 :INFO    ] - Creating path: ./network_datasets/notebook_tf_data
[18-12-23 18:37:03 :INFO    ] - Loading configuration loader.notebook_tf_data.json
[18-12-23 18:37:05 :INFO    ] - Dashboar

## IceNet UNet model

Running tensorflow UNet model as normal.

In [23]:
!icenet_train notebook_tf_data notebook_tf_testrun 42 -b 4 --epochs 10 --multiprocessing --max-queue-size 4 --workers 4 --n-filters-factor 0.5 --no-wandb

2023-12-18 18:39:36.902314: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 18:39:36.902393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 18:39:36.903666: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[18-12-23 18:39:45 :WARNING ] - Setting seed for best attempt at determinism, value 42
[18-12-23 18:39:45 :INFO    ] - Loading configuration dataset_config.notebook_tf_data.json
[18-12-23 18:39:45 :INFO    ] - Training dataset path: ./network_datasets/notebook_tf_data/south/train
[18-12-23 18:39:45 :INFO    ] - Validation dataset path: ./network_datasets/notebook

In [24]:
!./loader_test_dates.sh notebook_tf_data | tee testdates.csv

2020-04-01
2020-04-02


In [25]:
!icenet_predict -n 0.5 -t \
    notebook_tf_data notebook_tf_testrun example_south_tf_forecast 42 testdates.csv

2023-12-18 18:42:29.631755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 18:42:29.631832: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 18:42:29.633065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[18-12-23 18:42:35 :INFO    ] - Loading configuration ./dataset_config.notebook_tf_data.json
[18-12-23 18:42:35 :INFO    ] - Training dataset path: ./network_datasets/notebook_tf_data/south/train
[18-12-23 18:42:35 :INFO    ] - Validation dataset path: ./network_datasets/notebook_tf_data/south/val
[18-12-23 18:42:35 :INFO    ] - Test dataset path: ./network_datas

In [28]:
# Create netCDF files from npy prediction outputs.
!icenet_output -o results/predict example_south_tf_forecast notebook_tf_data testdates.csv

2023-12-18 18:44:49.074610: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 18:44:49.074691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 18:44:49.075920: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[18-12-23 18:44:54 :INFO    ] - Loading configuration ./dataset_config.notebook_tf_data.json
[18-12-23 18:44:54 :INFO    ] - Training dataset path: ./network_datasets/notebook_tf_data/south/train
[18-12-23 18:44:54 :INFO    ] - Validation dataset path: ./network_datasets/notebook_tf_data/south/val
[18-12-23 18:44:54 :INFO    ] - Test dataset path: ./network_datas

In [29]:
from icenet.plotting.video import xarray_to_video as xvid
from icenet.data.sic.mask import Masks

ds = xr.open_dataset("results/predict/example_south_tf_forecast.nc")
land_mask = Masks(south=True, north=False).get_land_mask()
ds.info()

xarray.Dataset {
dimensions:
	time = 2 ;
	yc = 432 ;
	xc = 432 ;
	leadtime = 7 ;

variables:
	int32 Lambert_Azimuthal_Grid() ;
		Lambert_Azimuthal_Grid:grid_mapping_name = lambert_azimuthal_equal_area ;
		Lambert_Azimuthal_Grid:longitude_of_projection_origin = 0.0 ;
		Lambert_Azimuthal_Grid:latitude_of_projection_origin = -90.0 ;
		Lambert_Azimuthal_Grid:false_easting = 0.0 ;
		Lambert_Azimuthal_Grid:false_northing = 0.0 ;
		Lambert_Azimuthal_Grid:semi_major_axis = 6378137.0 ;
		Lambert_Azimuthal_Grid:inverse_flattening = 298.257223563 ;
		Lambert_Azimuthal_Grid:proj4_string = +proj=laea +lon_0=0 +datum=WGS84 +ellps=WGS84 +lat_0=-90.0 ;
	float32 sic_mean(time, yc, xc, leadtime) ;
		sic_mean:long_name = mean sea ice area fraction across ensemble runs of icenet model ;
		sic_mean:standard_name = sea_ice_area_fraction ;
		sic_mean:short_name = sic ;
		sic_mean:valid_min = 0 ;
		sic_mean:valid_max = 1 ;
		sic_mean:ancillary_variables = sic_stddev ;
		sic_mean:grid_mapping = Lambert_Azimuth

In [30]:
forecast_date = ds.time.values[0]
print(forecast_date)

2020-04-01T00:00:00.000000000


In [31]:
fc = ds.sic_mean.isel(time=0).drop_vars("time").rename(dict(leadtime="time"))
fc['time'] = [pd.to_datetime(forecast_date) \
              + dt.timedelta(days=int(e)) for e in fc.time.values]

anim = xvid(fc, 15, figsize=4, mask=land_mask)
HTML(anim.to_jshtml())

INFO:root:Inspecting data
INFO:root:Initialising plot
INFO:root:Animating
INFO:root:Not saving plot, will return animation
INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


## Version
- IceNet Codebase: v0.2.7_dev